In [72]:
using Plots
using GLM
using DataFrames
using Statistics

In [38]:
# Important variables
threadsUsed = [1, 2, 4, 6, 8, 10, 12, 14, 16, 20, 24, 28, 32, 36, 40, 44, 48, 56, 64, 72, 80, 88, 96]
NRUNS = 5

# Ideal strong scaling
ideal = 1:maximum(threadsUsed)

1:96

In [39]:
# Function to extract data from output
function extract_data(filename::AbstractString)
    data = zeros(length(threadsUsed), NRUNS)
    
    current_thread = 0
    
    i = 0
    j = 1
    
    open(filename, "r") do file
        for line in eachline(file)
            if occursin("Threads", line)
                current_threads = filter(isdigit, line)
                i+=1
                j =1
            elseif occursin("Parallel RunTime  :", line)
                time = parse(Float64, split(line, "Parallel RunTime  :")[end])
                data[i, j] = time
                j+=1
            end
        end
    end
    return data
end

extract_data (generic function with 1 method)

In [40]:
data = extract_data("Loop-52021511.out")

23×5 Matrix{Float64}:
 5.40402   5.78349   5.67836   5.72604   5.75979
 3.71012   3.45383   3.67912   3.9087    3.7059
 2.33203   2.62806   2.89657   2.77943   2.78902
 1.79887   1.77901   1.57121   1.76906   1.66402
 1.26967   1.27299   1.22658   1.2658    1.17902
 1.01698   1.16325   1.18449   0.950213  1.12498
 0.862718  1.00033   0.98447   0.834331  0.919111
 0.684131  0.722687  0.799322  0.80026   0.696138
 0.585593  0.436179  0.757186  0.734628  0.661803
 0.53534   0.594033  0.541399  0.536875  0.551029
 0.4336    0.483799  0.440117  0.526559  0.476939
 0.345795  0.440032  0.44598   0.420391  0.383209
 0.378208  0.369546  0.312055  0.320638  0.366132
 0.364865  0.266409  0.30606   0.359589  0.339796
 0.264658  0.296894  0.297614  0.316606  0.308187
 0.223793  0.255957  0.290732  0.253399  0.239835
 0.241721  0.286831  0.234794  0.249318  0.279
 0.186681  0.251059  0.219269  0.22423   0.18057
 0.194519  0.174971  0.219412  0.195339  0.197545
 0.147948  0.173434  0.52073   0.193646

In [41]:
median_runtime = median(data, dims=2)
average_runtime = mean(data, dims=2)

23×1 Matrix{Float64}:
 5.670338600000001
 3.6915356000000004
 2.685023
 1.7164346000000001
 1.2428134
 1.08798
 0.920193
 0.7405076
 0.6350777999999999
 0.5517352000000001
 0.4722028
 0.40708140000000004
 0.3493158
 0.3273438
 0.2967918
 0.2527432
 0.2583328
 0.2123618
 0.1963572
 0.24353919999999998
 0.16867000000000001
 0.14700960000000002
 0.1545568

In [42]:
median_baseline_omp = median_runtime[1]
println(median_baseline_omp)
average_baseline_omp = average_runtime[1]
println(average_baseline_omp)

5.726043
5.670338600000001


In [43]:
# sequential version !!! (this is not omp with one thread)
sequential_runtimes = [7.795667, 5.703793, 6.582873, 5.850235, 5.694249]
average_sequential = mean(sequential_runtimes)
println(average_sequential)
median_sequential = median(sequential_runtimes)
println(median_sequential)

6.325363399999999
5.850235


In [44]:
median_speedup_omp = median_baseline_omp ./ median_runtime
average_speedup_omp = average_baseline_omp ./ average_runtime

median_speedup_seq = median_sequential ./ median_runtime
average_speedup_seq = average_sequential ./ average_runtime

23×1 Matrix{Float64}:
  1.1155177576168023
  1.7134775566027314
  2.3557948665616637
  3.6851758872723717
  5.089551979404148
  5.813859997426422
  6.873952964215114
  8.541929076757617
  9.95998191087769
 11.464491299449442
 13.395438146491294
 15.538325750083395
 18.107865146666708
 19.323302900497882
 21.31246011513795
 25.026839099924345
 24.485328227774403
 29.78578727435913
 32.21355468503319
 25.97267051875016
 37.50141341080215
 43.02687307495564
 40.92581756351063

In [47]:
# Fit the data using GLM
Model_median_omp = lm(@formula(transpose(median_speedup_omp) ~ threadsUsed), DataFrame(median_speedup_omp=median_speedup_omp[:], threadsUsed=threadsUsed))
Model_average_omp = lm(@formula(transpose(average_speedup_omp) ~ threadsUsed), DataFrame(average_speedup_omp=average_speedup_omp[:], threadsUsed=threadsUsed))
Model_median_seq = lm(@formula(transpose(median_speedup_seq) ~ threadsUsed), DataFrame(median_speedup_seq=median_speedup_seq[:], threadsUsed=threadsUsed))
Model_average_seq = lm(@formula(transpose(average_speedup_seq) ~ threadsUsed), DataFrame(average_speedup_seq=average_speedup_seq[:], threadsUsed=threadsUsed))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(transpose(average_speedup_seq)) ~ 1 + threadsUsed

Coefficients:
────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────
(Intercept)  2.27029    0.799969    2.84    0.0099   0.606659   3.93391
threadsUsed  0.438218   0.0177889  24.63    <1e-16   0.401224   0.475212
────────────────────────────────────────────────────────────────────────

In [54]:
# Get the slope for the median speedup with OpenMP baseline model
slope_median_omp = coef(Model_median_omp)[2]

# Get the slope for the average speedup with OpenMP baseline model
slope_average_omp = coef(Model_average_omp)[2]

# Get the slope for the median speedup with sequential baseline model
slope_median_seq = coef(Model_median_seq)[2]

# Get the slope for the average speedup with sequential baseline model
slope_average_seq = coef(Model_average_seq)[2]

0.4382182641134646

In [49]:
Fit_median_omp = [coef(Model_median_omp)[2] * i + coef(Model_median_omp)[1] for i in ideal]
Fit_average_omp = [coef(Model_average_omp)[2] * i + coef(Model_average_omp)[1] for i in ideal]
Fit_median_seq = [coef(Model_median_seq)[2] * i + coef(Model_median_seq)[1] for i in ideal]
Fit_average_seq = [coef(Model_average_seq)[2] * i + coef(Model_average_seq)[1] for i in ideal]

96-element Vector{Float64}:
  2.7085043360824095
  3.1467226001958744
  3.584940864309339
  4.023159128422804
  4.461377392536268
  4.8995956566497325
  5.337813920763198
  5.776032184876662
  6.214250448990127
  6.652468713103591
  7.0906869772170555
  7.52890524133052
  7.967123505443984
  ⋮
 39.51883852161344
 39.957056785726905
 40.395275049840365
 40.83349331395383
 41.2717115780673
 41.70992984218076
 42.14814810629422
 42.58636637040769
 43.024584634521155
 43.46280289863462
 43.90102116274808
 44.339239426861546

In [66]:
scatter(threadsUsed, median_speedup_omp, xlabel="Number of Threads", ylabel="Speedup", label="Measured Mean Runtime, Base OMP=1")
plot!(ideal, label="Ideal (1.0)")
plot!(Fit_median_omp, label="Fit ($slope_median_omp)")
savefig("median_speedup_omp_plot.pdf")

"/home/timo/ETH/FS24/HPCL24/Project02/project02_skeleton_codes/loop-dependencies/median_speedup_omp_plot.pdf"

In [67]:
scatter(threadsUsed, average_speedup_omp, xlabel="Number of Threads", ylabel="Speedup", label="Measured Mean Runtime, Base OMP=1")
plot!(ideal, label="Ideal (1.0)")
plot!(Fit_average_omp, label="Fit ($slope_average_omp)")
savefig("average_speedup_omp_plot.pdf")

"/home/timo/ETH/FS24/HPCL24/Project02/project02_skeleton_codes/loop-dependencies/average_speedup_omp_plot.pdf"

In [68]:
scatter(threadsUsed, median_speedup_seq, xlabel="Number of Threads", ylabel="Speedup", label="Measured Mean Runtime, Base sequential")
plot!(ideal, label="Ideal (1.0)")
plot!(Fit_median_seq, label="Fit ($slope_median_seq)")
savefig("median_speedup_seq_plot.pdf")

"/home/timo/ETH/FS24/HPCL24/Project02/project02_skeleton_codes/loop-dependencies/median_speedup_seq_plot.pdf"

In [69]:
scatter(threadsUsed, average_speedup_seq, xlabel="Number of Threads", ylabel="Speedup", label="Measured Mean Runtime, Base sequential")
plot!(ideal, label="Ideal (1.0)")
plot!(Fit_average_seq, label="Fit ($slope_average_seq)")
savefig("average_speedup_seq_plot.pdf")

"/home/timo/ETH/FS24/HPCL24/Project02/project02_skeleton_codes/loop-dependencies/average_speedup_seq_plot.pdf"